In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd '/content/drive/MyDrive/CV8160 S24/HW1'

In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from pathlib import Path
from PIL import Image
import warnings
warnings.filterwarnings("ignore")
plt.ion()   # interactive mode

In [ ]:
class hw1(Dataset)
  def __init__(self, root_dir)
  self.root = Path(root_dir)
  self.df = pd.DataFrame(columns=)